In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.2 MB/s eta 0:00:00


In [ ]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 27.9 MB/s eta 0:00:00


In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 30.1 MB/s eta 0:00:00


In [ ]:
pip install faiss-cpu langchain transformers sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 79.3 MB/s eta 0:00:00


In [ ]:
import json
from pathlib import Path
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
import random

class EmbeddingManager:
    def __init__(self, model_name):
        self.embeddings = HuggingFaceEmbeddings(model_name=model_name)

    def get_embeddings(self):
        return self.embeddings

class VectorSpaceManager:
    def __init__(self, embedding_manager):
        self.embedding_manager = embedding_manager
        self.embeddings = self.embedding_manager.get_embeddings()

    def create_vector_space(self, documents):
        print("Embedding documents...")
        vector_store = FAISS.from_documents(documents, self.embeddings)
        return vector_store

    def save_vector_space(self, vector_store, save_path):
        print(f"Saving vector space to {save_path}...")
        vector_store.save_local(save_path)
        print(f"Finished!")

    def load_vector_space(self, save_path):
        print(f"Loading vector space from {save_path}...")
        return FAISS.load_local(save_path, self.embeddings, allow_dangerous_deserialization=True)

class DataLoader:
    def __init__(self, json_file_path):
        self.json_file_path = json_file_path

    def load_data(self):
        data = json.loads(Path(self.json_file_path).read_text())
        return data

    def create_documents(self, length=None):
        data = self.load_data()
        conversations = data['conversations']

        if length is None:
            length = len(conversations)

        documents = [
            Document(
                page_content=self.get_page_content(item),
                metadata=item
            )
            for item in conversations[:length]
        ]
        return documents

    def get_page_content(self, item):
        raise NotImplementedError("Subclasses must implement get_page_content method")

#Concatenate the Intent, User Queries and Bot Responses into a single string
class ConversationDataLoader(DataLoader):
    def get_page_content(self, item):
        return f"Intent: {item['intent']} | User Queries: {' '.join(item['user_queries'])} | Bot Responses: {' '.join(item['bot_response'])}"
#Load create and save vector space
def process_data(json_file_path, model_name, save_path, data_loader_class, length=None):
    embedding_manager = EmbeddingManager(model_name)
    vector_space_manager = VectorSpaceManager(embedding_manager)

    data_loader = data_loader_class(json_file_path)
    documents = data_loader.create_documents(length=length)
    vector_store = vector_space_manager.create_vector_space(documents)
    vector_space_manager.save_vector_space(vector_store, save_path)

#Performs Similarity search in the vector store using user query and extract the first response
def converse_with_bot(user_query, vector_store):
    search_results = vector_store.similarity_search(user_query, k=1)
    if search_results:
        response_list = search_results[0].metadata['bot_response']
        response = random.choice(response_list)
        return response
    else:
        return "I'm sorry, I couldn't find a relevant response."

if __name__ == "__main__":
    json_file_path = '/content/JSONConversation.json'
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    save_path = '/content/migration_vector_space/'

    process_data(json_file_path, model_name, save_path, ConversationDataLoader, 100)

    vector_store = VectorSpaceManager(EmbeddingManager(model_name)).load_vector_space(save_path)

    while True:
        user_query = input("User: ")
        if user_query.lower() in ['exit', 'quit']:
            break
        bot_response = converse_with_bot(user_query, vector_store)
        print(f"Assistant: {bot_response}")


<ipython-input-7-aa6c4654af74>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding documents...
Saving vector space to /content/migration_vector_space/...
Finished!
Loading vector space from /content/migration_vector_space/...
User: Hello
Assistant: Hello! What would you like to do today?
User: I want to migrate my data
Assistant: I’m happy to help! Where are you moving your data from?
User: mysql
Assistant: What table names are we looking at in the source database?
User: source table name is usertable
Assistant: What are the tables in the source database?
User: I already gave you source database what else do yo uneed
Assistant: Can you tell me the table names in the source?
User: what else do you need
Assistant: Is there any other connection data you'd like to enter?
User: yes
Assistant: u
User: I want to connect on port 8080
Assistant: Any other connection details we should apply?
User: No
Assistant: h
User: I also want to provide destination details
Assistant: a
User: I want to move my data from Mysql to postgres
Assistant: Understood. Are there any chan

KeyboardInterrupt: Interrupted by user